In [ ]:
# from libraries
import pandas as pd
import dask.dataframe as dd
# from files
import params
import auth
import get_data_util
import get_sp500_util as sp500
import get_dow_jones_util as dj

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import pandas as pd
import dask.dataframe as dd
from io import StringIO
import quandl
import time
import boto3
import datetime

# from files:
import auth
import params

In [ ]:
ticker_list = dj.get_dow_jones(url=dj.url, headers=auth.user_agent)
quandl_parameters = [a for a in dir(params) if ("_parameters" in a)]

In [ ]:
stock_info_list = []
alpha_vantage_base = "https://www.alphavantage.co"  # url base
param_dict_names = [name for name in dir(params) if ('_params' in name)]
param_dict_list = [getattr(params, name) for name in param_dict_names]

In [ ]:
param_dict_names

In [ ]:
for stock_symbol in ticker_list:
    stock_indicators = []
    for i in range(len(param_dict_list)):
        session = requests.Session()
        retry = Retry(connect=5, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount("http://", adapter)
        session.mount("https://", adapter)
        param_dict_list[i]["symbol"] = stock_symbol
        r = session.get(alpha_vantage_base + '/query', params=param_dict_list[i], headers=auth.user_agent)
        df = pd.read_csv(StringIO(r.content.decode('utf-8')), engine="python").iloc[1:,:]
        df.columns = [param_dict_list[i]["function"] + "_" + str(column) for column in df.columns]
        df.set_index(df.columns[0], inplace=True)
        if param_dict_list[i]["function"] == "TIME_SERIES_DAILY_ADJUSTED":
            df["stock_symbol"] = stock_symbol
        stock_indicators.append(df)
        print("Successfully extracted {}!".format(stock_symbol+"_"+param_dict_names[i].replace("_params","")))
    stock_info = pd.concat(stock_indicators, axis=1)
    stock_info = stock_info.loc[:,~stock_info.columns.duplicated()]
#    stock_info_list.append(stock_info)
#concat_data = pd.concat(stock_info_list, axis=0, sort=False, ignore_index=True)

In [ ]:
len(stock_indicators)

In [ ]:
stock_info

In [ ]:
len(stock_info.columns)

In [ ]:
list(stock_info_list[0].columns)

In [ ]:
list(stock_info_list[2].columns)

In [ ]:
quandl.ApiConfig.api_key = auth.apikey_quandl
indicator_data_list = []

In [ ]:
for param in quandl_parameters:
        dict_ = getattr(params, param)
        data = quandl.get("{}/{}".format(dict_["database_code"], dict_["dataset_code"]),
                          start_date=dict_["start_date"], end_date=dict_["end_date"])
        data.columns = [param.split("_")[0] + "_" + str(column) for column in data.columns]
        indicator_data_list.append(data)
indicator_data = pd.concat(indicator_data_list, axis=1, sort=False)

In [ ]:
indicator_data

In [ ]:
stock_info

In [ ]:
stock_info.join(indicator_data)